# STAT 301 Group 19 - Final Project Report
Authors: Andy Hong, Benedictus Harley Kartawidjaja, Dakshish Raj Paulvannan, Nathan Chang

## 1. Introduction

The primary goal of our analysis is to analyze potential predictors to determine the odds of an employee staying or leaving a company, which will be conducted using the Employees dataset. Understanding the factors behind employee retention is crucial for companies to measure performance, reduce costs, and to continue improving workplace practices (Das et al, 2013).

The Employees dataset consists of anonymized information about employees in a company, including: education level, professional history, salary, demographics, and information regarding their work with the company. This data was collected by the company Human Resources department and contributed to Kaggle, a community offering data science resources and tools.

### Overview of variables:
- **Education** *(categorical)*: chr representing the highest level of education attained by the employee.

- **JoiningYear** *(numerical)*: dbl representing the year a given employee first started working for the company.

- **City** *(categorical)*: chr representing the city in which an employee works for the company.

- **PaymentTier** *(categorical)*: dbl representing a bin containing an employee's salary.

- **Age** *(numerical)*: dbl representing the age of employees working at a company.

- **Gender** *(categorical)*: chr representing the sex of employees working at a company.

- **EverBenched** *(binary)*: chr representing whether or not an employee has been benched, ie. had a period of time without work assigned to them.

- **ExperienceInCurrentDomain** *(categorical)*: dbl representing the years of experience an employee has in the area of their role prior to joining the company.

- **LeaveOrNot** *(binary)*: dbl representing whether an employee left of not.

To select a model, we'd analyze the relationship of these predictors with our response, LeaveOrNot, and their relationships with each other to determine the presence of interaction. After completing our analysis, we would then train a logistic model using these predictors to predict an employee's odds of leaving using a testing set. A logistic model was selected as our response is a binary variable.

Our data supports this by providing information on all of these predictors, however, we will need to transform the data by pre-processing and dropping variables we do not wish to include.

**We excluded the following predictors:**
- **Gender**: We chose to drop gender as it isn't inclusive as when the data was collected, Gender was presented as a binary option when in practice, many people identify as genders beyond the binary. Additionally current literature suggests that gender inequality leads to inaccurate results on employee retention (Lijster et al, 2023).

- **JoiningYear**: We elected to exclude this as we felt that it wouldn't be able to provide much context without other information that we did not have access to. Ex. If we wanted to use JoiningYear to determine how long an employee has worked at the company, we are unable to do so without accessing information on when an employee left the company (if applicable). This approach is also flawed in that we won’t know how long an employee marked as staying is planning on continuing at the company (ex. There is no difference between an employee leaving in 1 day vs 1 year).

## 2. Methods and Results

### a. Exploratory Data Analysis

In [2]:
# Attaching packages. Source: Course worksheets
library(plyr)
library(dplyr)
library(dbplyr)
library(tidyverse)
library(cowplot)
library(ggplot2)
library(reshape2)
library(repr)
library(infer)
library(broom)
library(AER)
library(modelr)
library(glmnet)
library(tidymodels)
library(mltools)
library(caret)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    mas

In [3]:
#Main Developer: Benedictus Harley Kartawidjaja
#Contributor: Nathan Chang

employee <- read.csv("data/Employee.csv", header = TRUE)

employee <- employee |>
    select(-Gender, -JoiningYear)

head(employee)

,Education,City,PaymentTier,Age,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,Bachelors,Bangalore,3,34,No,0,0
2,Bachelors,Pune,1,28,No,3,1
3,Bachelors,New Delhi,3,38,No,2,0
4,Masters,Bangalore,3,27,No,5,1
5,Masters,Pune,3,24,Yes,2,1
6,Bachelors,Bangalore,3,22,No,0,0


**Figure 1: Preview of Employee Data (First 6 Rows)**

**Figure 1**\
Preview of Employee Data (First 6 Rows)

In [ ]:
#Main Developer: Benedictus Harley Kartawidjaja

unique(employee$LeaveOrNot) # 2 levels (0, 1)
unique(employee$ExperienceInCurrentDomain) # 8 levels (0 - 7)
unique(employee$EverBenched) #  2 levels ("No" and "Yes")
unique(employee$PaymentTier) # 3 levels (1, 2, and 3)
unique(employee$City) # 3 levels ("Bangalore", "Pune", and "New Delhi")
unique(employee$Education) # 3 levels ("Bachelors", "Masters", and "PHD")

**Figure 2: Unique Values in Selected Employee Data Columns**

In [ ]:
#Main Developer: Benedictus Harley Kartawidjaja

employee <- employee |>
    mutate(EverBenched = ifelse(EverBenched == "Yes", 1, 0),   # EverBenched into 0 for "No" and 1 for "Yes"
           City = ifelse(City == "Pune", 1,
                        ifelse(City == "New Delhi", 2, 0)),   # "Bangalore" is 0, "Pune" is 1, "New Delhi" is 2
           Education = ifelse(Education == "Masters", 1,
                             ifelse(Education == "PHD", 2, 0)))   # "Bachelors" is 0, "Masters" is 1, "PHD" is 2
head(employee)

**Figure 3: Transformation of 'EverBenched', 'City', and 'Education' Columns into Numeric Values**

### b. Methods and Plan

### Making a Heatmap

Using a **heatmap**, we evaluated the correlation of all our predictors with response and each other. We then used the results of our heatmap to guide whether we include or exclude a predictor in our analysis. 

In [ ]:
#Main Developer: Nathan Chang

corr_matrix_employee <- 
   employee %>%
   cor() %>%
   as_tibble(rownames = 'var1') %>%
   pivot_longer(-var1, names_to = "var2", values_to = "corr")

head(corr_matrix_employee)

**Figure 4: Correlation Matrix for Employee Data (First 6 Rows)**

In [ ]:
#Main Developer: Nathan Chang

options(repr.plot.width = 10, repr.plot.height = 6) 

employee_processed_plot <- 
  corr_matrix_employee %>%
  ggplot(aes(var1, var2)) +
  geom_tile(aes(fill = corr), color = "white") +
  scale_fill_distiller("Correlation Coefficient \n",
      palette =  "YlOrRd",
      direction = 1, 
      limits = c(-1, 1)
    ) +
    labs(x = "", y = "") +
    theme_minimal() +
    theme(
        axis.text.x = element_text(angle = 45, vjust = 1, size = 14, hjust = 1),
        axis.text.y = element_text(vjust = 1, size = 14, hjust = 1),
        legend.title = element_text(size = 18),
        legend.text = element_text(size = 12),
        legend.key.size = unit(1.5, "cm")
    ) +
    coord_fixed() + geom_text(aes(var1, var2, label = round(corr, 2)), color = "black", size = 6)

employee_processed_plot

**Figure 5: Heatmap of Correlation Coefficients Between Employee Data Variables**

### Interpreting the Heatmap:

Based on the results of our heatmap, none of our predictors are highly correlated with each other (corr ranges from -0.14 to 0.39). This suggests that our predictors don't interact with each other very much. This suggests that an additive model would be appropriate given the lack of interaction between predictors.

Additionally, our predictors (when considered individually) aren't very correlated with our response, LeaveOrNot (corr ranges from -0.05 to 0.2). When considered together, it may be possible to develop a more reliable model. As such, we elected to not exclude any predictors.

### Our Plan:

To approach our question, we will conduct **logistic regression** as our response is a binary variable. This will be fit for a **predictive model**.

As seen from the EDA and Correlation Heatmap above, we had a wide selection of predictors to select from. To constrain variables that are deemed relevant to our topic, we split our dataset and performed **LASSO Regularization** to shrink our model before performing regression.  We then evaluated our model.

### Feature Selection:

The goal for our LASSO regularization was to find the optimum number of predictors that produces the highest average AUC. In doing so, the LASSO method reduces the number of predictors we used in our model to improve our model’s interpretability. 

Cross-fold validation was used to select the number of parameters as it reduces the risk of overfitting, tunes parameters and selects those that generalize well to new data, and reduces variability.

We then split our data into a 60-40 split and dropped `Gender` and `JoiningYear` as per the reasons in the intro.

In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

## load data
employee  <- read.csv("data/Employee.csv")

## split into training and testing
split <- initial_split(employee, prop = 0.6) 

employee_training <- training(split) %>%
    select(-Gender, -JoiningYear)
employee_testing <- testing(split) %>%
    select(-Gender, -JoiningYear)

model_matrix_X_train <- 
    model.matrix(LeaveOrNot ~ ., employee_training)[,-1]

matrix_Y_train <- 
    as.matrix(employee_training$LeaveOrNot, ncol = 1)

With our prepared training data, we used `cv.glmnet` to find the optimal $\lambda$ value that provides the largest average AUC and used 5-fold cross validation.

In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

employee_cv_lambda_LASSO <- 
  cv.glmnet(
  x = model_matrix_X_train, y = matrix_Y_train,
  alpha = 1,
  family = 'binomial',
  type.measure = 'auc',
  nfolds = 5)

employee_cv_lambda_LASSO

**Figure 6: Cross-Validation Results for LASSO Logistic Regression Model on Employee Data**

In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

options(repr.plot.width = 10, repr.plot.height = 6)

plot(employee_cv_lambda_LASSO, main = "Cross-Validation with LASSO\n\n")

**Figure 7: LASSO Cross-Validation Plot for Employee Data**

The numbers across the top of the graph indicate the number of predictors (value of $\lambda$) used. For our case, we used the second line to the right which represents $\hat{\lambda}_{\text{1SE}}$ from figure 2, as our goal is to improve model interpretibility and $\hat{\lambda}_{\text{1SE}}$ has less covarites. 


In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

employee_cv_lambda1se_LASSO_value <- round(employee_cv_lambda_LASSO$lambda.1se, 4)
employee_cv_lambda1se_LASSO_value

We then used this value for LASSO to determine which predictors to select using `glmnet`.

In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

employee_LASSO <- glmnet(
x = model_matrix_X_train, y = matrix_Y_train,
  alpha = 1,
  family = 'binomial',
  lambda = employee_cv_lambda1se_LASSO_value
)

coef(employee_LASSO)

**Figure 8: Coefficients of LASSO Logistic Regression Model for Employee Data**

Our LASSO suggested we use `EducationMasters`, `CityNew Delhi`, `CityPune`, `PaymentTier`, `Age`,  and `EverBenchedYes` as our parameters. We chose to use the base predictors, rather than subsets of our variables (`EverBenchedYes` rather than just `EverBenched`) to improve simplicity and model interpretability. 

### Testing the Model

We fitted a  logistic model based on the covariates found above using the training data. Then, we used the trained model to predict the testing data displayed in a confusion matrix.

In [ ]:
# Main Developer: Andy Hong

set.seed(1234)

employee_logistic_model <- glm(LeaveOrNot ~ Education + City + PaymentTier + Age + EverBenched, 
                               data = employee_training, family = 'binomial')

employee_pred_class <- round(predict(employee_logistic_model, newdata = employee_testing, type = 'response'))

employee_confusion_matrix <- 
    confusionMatrix(
    data = as.factor(employee_pred_class),
    reference = as.factor(employee_testing$LeaveOrNot),
    positive = '1'
)
employee_confusion_matrix

**Figure 9: Confusion Matrix for Logistic Regression Model Predicting Employee Leave Status**

Our model yielded an accuracy level of 68.89%, which is the percentage of correct predictions that our model (which was trained on `employee_training`) made against the `employee_testing` data (unseen observations). Our model had a sensitivity of 0.3418, with the desired rate being closer to 1. Our model had a Cohen's Kappa score of 0.2278, with the desired score being closer to 1.

The p-value in the confusion matrix uses a one-sided test if the accuracy is better than having no information (NIR = No Information Rate). Our model reported a p-value of 0.008504, which is less than 0.05. This indicates our results are statistically significant at the 0.05 significance level. What about covariates? 

As seen above, p-values for all covariates used (hence excluding EducationPHD) were less than a significance level of 0.05. 

As a model assessment, we can say that predicting employee retention based off of these covariates will achieve an accurate result around 68.89% of the time.s. Let's look at how the testing data was collected: the employee data set was anonymized in different demographics, and was found on a public domain. Furthermore, there is mainly quantitative data collected, with no information on workplace conditions or culture. Therefore our model can be assessed as good for what we were given.


In [ ]:
# Main Developer: Dakshish Raj Paulvannan
# Contributors: Andy Hong (name)

tidy(employee_logistic_model)

**Figure 10: Summary of Logistic Regression Model Coefficients for Employee Data**

## 3. Discussion

### Summary of Findings

The primary goal of our analysis is to analyze potential predictors to determine the odds of an employee staying or leaving a company. Using logistic regression with LASSO regularization, we have identified 5 significant predictors: Education, PaymentTier, City, Age, and EverBenched.

### Key Findings

**Education**: Our analysis suggests employees with higher education levels have a higher turnover rate. This might be due to employees having higher demand compared to employees with just a Bachelors.

**City**: Our model suggests location influences employee retention. However, an interesting find is that employees are significantly more likely to leave their job in a city like Pune compared to New Delhi. This may be caused by differences in work culture or the availability of roles with higher salaries (Sheridan 1993).

**PaymentTier**: Employees in higher payment tiers have a lower turnover rate. This aligns with the common understanding that better compensation incentivizes employees to remain at the company, increasing retention. 

**Age**: Older employees are more likely to remain at the company. Shapiro et al. (2022) investigates how years of service influence retention rates, noting that younger employees get burnout more easily, causing them to have a higher probability of leaving.

**EverBenched**: Employees who have experienced being benched (period of time without work) are more likely to leave. These extended periods without work may have create

### Implications and Impact

By understanding the factors that significantly contribute to the retention of the employees, companies can develop targeted strategies to improve retention. For example:
- **Enhance employee compensation packages**: Improving the salary structures, especially for lower payment tiers, could help reduce turnover. Companies can give bonuses to employees with better education, incentivising them to stay in the company long term.

- **Career progression opportunities**: Ensuring a clear path for progression such as guaranteed promotion especially for employees with a requisite education level after a set period of time or completed work may also incentivize employees to stay longer.

- **Location**: The company may consider focusing its operations in a location with a higher retention rate such as New Delhi instead of Pune where the retention rate is lower, or further investigate the discrepancy in retention rate between locations.

- **Workload**: The company should ensure that every employee has consistent work assignments as benching can negatively impact employee retention. 

### Model Improvements

While the logistic regression model with LASSO regularization was able to provide valuable insights, there are several ways it could be improved:

- **Inclusion of More Predictors Incorporating** - additional predictors can enhance the model's comprehensiveness and predictive power. Potential additional predictors include:
    - **Role**: Different roles may have varying turnover rates due to differences in job satisfaction, career progression opportunities, and market demand.
    - **Performance Metrics**: Employee performance scores, such as key performance indicators (KPIs), can provide insights into employee satisfaction and engagement.
    - **Work-Life Balance**: Metrics or survey data related to work-life balance can help identify if employees are leaving due to burnout or work-related stress.
    
- **Advanced Models** - Exploring more sophisticated models can improve predictive performance and provide deeper insights:
    - **Random Forest**: An ensemble learning method that can capture non-linear relationships and interactions between predictors.

### Future Questions and Research

The findings from this study open up several avenues for future research, each aiming to deepen the understanding of employee turnover and develop more effective retention strategies. Some potential questions and research directions include:
- How does organizational and work culture influence employee retention? Future research could explore the specific aspects of organizational culture that contribute to higher or lower retention rates. 

- How do career development opportunities affect employee retention? Research could focus on the relationship between career development programs (training, mentorship, and promotion opportunities) and employee retention rates.

- How do employee retention patterns differ across countries and cultures? Comparative studies across different geographic regions and cultures can reveal unique turnover patterns and inform globally applicable retention strategies. 

## 4. References
Das, Bidisha Lahkar, and Mukulesh Baruah. "Employee retention: A review of literature." *Journal of business and management* 14.2 (2013): 8-16.

Elmetwally, Tawfik. “Employee Dataset.” *Kaggle*, Public Domain, 6 Sept. 2023, www.kaggle.com/datasets/tawfikelmetwally/employee-dataset/data.

Lijster, Marrit, and Georges El Hajal. “Beyond gender bias: Factors affecting female employee retention and advancement in the Dutch hotel sector.” *Research in Hospitality Management*, vol. 13, no. 2, 2023, pp. 83–92, https://doi.org/10.1080/22243534.2023.2277515. 

Shapiro, Daniel, et al. “The Seniority Swoop.” *Nursing Administration Quarterly*, vol. 46, no. 1, 2022, pp. 60–71, https://doi.org/10.1097/NAQ.0000000000000502. 

Sheridan, John E. "Organizational culture and employee retention." *Academy of management Journal* 35.5 (1992): 1036-1056.

Anand, Amitabh, Jessica Doll, and Prantika Ray. "Drowning in silence: a scale development and validation of quiet quitting and quiet firing." *International Journal of Organizational Analysis* 32.4 (2024): 721-743.